#### import necessary libraries

In [17]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi
import seaborn as sns
import matplotlib.pyplot as plt 

#### read in the data

In [2]:
data1 = pd.read_csv('gapminder2.csv', low_memory=False)

#### remove unnecessary columns and make a copy of the subdata

In [3]:
data2 = data1[["continent", "country", "breastcancerper100th", "urbanrate", "incomeperperson", "breastcancernbdeaths"]]
data = data2.copy()

#### remove missing values(in my case '0' values)

In [4]:
data= data.replace(0, np.NaN)
data = data.dropna()

In [5]:
print(len(data))
print(len(data.columns))

165
6


#### Change the data type for chosen variables

In [6]:
data['breastcancerper100th'] = pd.to_numeric(data['breastcancerper100th'])
data['breastcancernbdeaths'] = pd.to_numeric(data['breastcancernbdeaths'])
data['incomeperperson'] = pd.to_numeric(data['incomeperperson'])
data['urbanrate'] = pd.to_numeric(data['urbanrate'])


#### Create variable quartiles and calculate frequency in bins 

In [7]:
data['urbanratepercent'] =pd.cut(data.urbanrate,4,labels=['0-25%','26-50%','51-74%','75-100%'])
urban_freq = pd.concat(dict(counts = data["urbanratepercent"].value_counts(sort=False, dropna=False),
                                   percentages = data["urbanratepercent"].value_counts(sort=False, dropna=False,
                                                                                       normalize=True)),
                            axis=1)
print("Frequency distribution - urban rate:\n", urban_freq)

Frequency distribution - urban rate:
          counts  percentages
0-25%        32     0.193939
26-50%       42     0.254545
51-74%       61     0.369697
75-100%      30     0.181818


In [8]:
print('Income per person in categories')
data['incomelabel'] =pd.cut(data.incomeperperson,4,labels=['low','medium','high','very high'])
income_freq = pd.concat(dict(counts = data["incomelabel"].value_counts(sort=False, dropna=False),
                                   percentages = data["incomelabel"].value_counts(sort=False, dropna=False,
                                                                                       normalize=True)),
                            axis=1)
print("Frequency distribution - income per person:\n", income_freq)

Income per person in categories
Frequency distribution - income per person:
            counts  percentages
high           12     0.072727
low           134     0.812121
medium         16     0.096970
very high       3     0.018182


In [9]:
data.head()

,continent,country,breastcancerper100th,urbanrate,incomeperperson,breastcancernbdeaths,urbanratepercent,incomelabel
1,Europe,Albania,57.4,46.72,1914.996551,300,26-50%,low
2,Africa,Algeria,23.5,65.22,2231.993335,2019,51-74%,low
3,Africa,Angola,23.1,56.70,1381.004268,654,51-74%,low
4,South America,Argentina,73.9,92.00,10749.419238,5362,75-100%,low
5,Asia,Armenia,51.6,63.86,1326.741757,561,51-74%,low


#### create a subset to include the 2 variables(1 categorical + 1 numerical) we want to analyse 

In [10]:
sub1 = data[['breastcancerper100th', 'urbanratepercent']]

#### using ols function for calculating the F-statistic and associated p value

In [11]:
model1 = smf.ols(formula='breastcancerper100th ~ C(urbanratepercent)', data=sub1)
results1 = model1.fit()
print (results1.summary())

                             OLS Regression Results                             
Dep. Variable:     breastcancerper100th   R-squared:                       0.328
Model:                              OLS   Adj. R-squared:                  0.316
Method:                   Least Squares   F-statistic:                     26.25
Date:                  Sun, 22 Oct 2017   Prob (F-statistic):           7.10e-14
Time:                          13:28:14   Log-Likelihood:                -718.30
No. Observations:                   165   AIC:                             1445.
Df Residuals:                       161   BIC:                             1457.
Df Model:                             3                                         
Covariance Type:              nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------

#### Since our p value is so small, 0.0000000000000710, we can safely reject the null hypothethis and accept that there is an association between the percent of urbanisation of countries and the number of breast cancer cases.

In [14]:
print ('means for breast cancer cases by urbanisation level')
m2= sub1.groupby('urbanratepercent').mean()
print (m2) 

means for breast cancer cases by urbanisation level
                  breastcancerper100th
urbanratepercent                      
0-25%                        21.643750
26-50%                       29.178571
51-74%                       40.131148
75-100%                      61.430000


In [15]:
print ('standard deviations for breast cancer cases by urbanisation level')
sd2 = sub1.groupby('urbanratepercent').std()
print (sd2)

standard deviations for breast cancer cases by urbanisation level
                  breastcancerper100th
urbanratepercent                      
0-25%                         8.567491
26-50%                       14.904469
51-74%                       20.438334
75-100%                      27.502992


#### run post-hoc test for ANOVA since our categorical variable has more than 2 levels

In [16]:
mc1 = multi.MultiComparison(sub1['breastcancerper100th'], sub1['urbanratepercent'])
res1 = mc1.tukeyhsd()
print(res1.summary())

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1  group2 meandiff  lower   upper  reject
----------------------------------------------
0-25%   26-50%  7.5348   -4.066 19.1356 False 
0-25%   51-74% 18.4874   7.6961 29.2787  True 
0-25%  75-100% 39.7862  27.2221 52.3504  True 
26-50%  51-74% 10.9526   1.0397 20.8655  True 
26-50% 75-100% 32.2514  20.4332 44.0697  True 
51-74% 75-100% 21.2989  10.2741 32.3236  True 
----------------------------------------------
